In [ ]:
!pip install --upgrade pip
!pip install -q textstat
!pip install scispacy
!pip install emot

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import gc
import os
import time
import math
import random
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from datetime import date
from transformers import *
from sklearn.metrics import *
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F



import warnings
warnings.filterwarnings("ignore")

import os
import gc
import re
import string
import folium
import textstat
from scipy import stats
from colorama import Fore, Back, Style, init

import math
import numpy as np
import scipy as sp
import pandas as pd

import random
import networkx as nx
from pandas import Timestamp

from PIL import Image
from IPython.display import SVG
from keras.utils import model_to_dot

import requests
from IPython.display import HTML

import seaborn as sns
from tqdm import tqdm
import matplotlib.cm as cm
import matplotlib.pyplot as plt

tqdm.pandas()

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

import transformers
import tensorflow as tf

from tensorflow.keras.callbacks import Callback
from sklearn.metrics import accuracy_score, roc_auc_score
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger

from tensorflow.keras.models import Model
from kaggle_datasets import KaggleDatasets
from tensorflow.keras.optimizers import Adam
from tokenizers import BertWordPieceTokenizer
from tensorflow.keras.layers import Dense, Input, Dropout, Embedding
from tensorflow.keras.layers import LSTM, GRU, Conv1D, SpatialDropout1D

from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import activations
from tensorflow.keras import constraints
from tensorflow.keras import initializers
from tensorflow.keras import regularizers

import tensorflow.keras.backend as K
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.activations import *
from tensorflow.keras.constraints import *
from tensorflow.keras.initializers import *
from tensorflow.keras.regularizers import *


from sklearn import metrics
from sklearn.utils import shuffle
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer  

import nltk
from textblob import TextBlob


from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from wordcloud import WordCloud, STOPWORDS
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import sent_tokenize, word_tokenize

stopword=set(STOPWORDS)

from scispacy.abbreviation import AbbreviationDetector
from emot.emo_unicode import UNICODE_EMO, EMOTICONS

lem = WordNetLemmatizer()
tokenizer=TweetTokenizer()

np.random.seed(0)

In [ ]:

# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train1 = pd.read_csv("../input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv")


train2 = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train.csv")
train2.toxic = train2.toxic.round().astype(int)

#train3 = pd.read_csv('../input/jigsaw-train-multilingual-coments-google-api/jigsaw-toxic-comment-train-google-es-cleaned.csv')


valid = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv')
test = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv')
sub = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv')


toxic = len(train2[['comment_text', 'toxic']].query('toxic==1'))

SAMPLE_SIZE = 10000

# Combine train1 with a subset of train2
train_cat = pd.concat([
    train1[['comment_text', 'toxic']],
    train2[['comment_text', 'toxic']].query('toxic==1'),
#    train3[['comment_text', 'toxic']].query('toxic==0'),
#    train3[['comment_text', 'toxic']].query('toxic==1'),   
    train2[['comment_text', 'toxic']].query('toxic==0').sample(n=(toxic+(toxic//3)), random_state=101)
]).sample(n=SAMPLE_SIZE).reset_index(drop=True) #restricting data to 600,000 records due to memory issue

test_data = test
train_data = train_cat
train_data_o  = train_cat

In [ ]:
train_data.describe()

In [ ]:
print(len(train_data))


In [ ]:
train_data.head()

In [ ]:
valid.head()

In [ ]:
valid['lang'].value_counts() 

In [ ]:
valid['toxic'].value_counts() 

In [ ]:
train_data_o['toxic'].value_counts() 


## Preprocessing

Making Class Balance (Toxic and Non-toxic)

In [ ]:
count_toxic_0, count_toxic_1 = train_data_o.toxic.value_counts()  


In [ ]:
df_toxic_0 = train_data_o[train_data_o['toxic'] == 0]
df_toxic_1 = train_data_o[train_data_o['toxic'] == 1]

In [ ]:
df_toxic_0_under = df_toxic_0.sample(count_toxic_1)
train_data_df = pd.concat([df_toxic_0_under, df_toxic_1], axis=0)

print('under-sampling:')
print(train_data_df.toxic.value_counts())

In [ ]:
# Number of words
train_data['num_words'] = train_data['comment_text'].apply(lambda x: len(str(x).split()))
#df_test['num_words'] = df_test['question_text'].apply(lambda x: len(str(x).split()))

# Number of capital_letters
train_data['num_capital_let'] = train_data['comment_text'].apply(lambda x: len([c for c in str(x) if c.isupper()]))
#df_test['num_capital_let'] = df_test['question_text'].apply(lambda x: len([c for c in str(x) if c.isupper()]))

# Number of special characters
train_data['num_special_char'] = train_data['comment_text'].str.findall(r'[^a-zA-Z0-9 ]').str.len()
#df_test['num_special_char'] = df_test['question_text'].str.findall(r'[^a-zA-Z0-9 ]').str.len()

# Number of unique words
train_data['num_unique_words'] = train_data['comment_text'].apply(lambda x: len(set(str(x).split())))
#df_test['num_unique_words'] = df_test['question_text'].apply(lambda x: len(set(str(x).split())))

# Number of numerics
train_data['num_numerics'] = train_data['comment_text'].apply(lambda x: sum(c.isdigit() for c in x))
#df_test['num_numerics'] = df_test['question_text'].apply(lambda x: sum(c.isdigit() for c in x))

# Number of characters
train_data['num_char'] = train_data['comment_text'].apply(lambda x: len(str(x)))
#df_test['num_char'] = df_test['question_text'].apply(lambda x: len(str(x)))

# Number of stopwords
train_data['num_stopwords'] = train_data['comment_text'].apply(lambda x: len([c for c in str(x).lower().split() if c in STOPWORDS]))

In [ ]:
def display_boxplot(_x, _y, _data, _title):
    sns.boxplot(x=_x, y=_y, data=_data)
    plt.grid(True)
    #plt.tick_params(axis='x', which='major', labelsize=15)
    plt.title(_title,fontsize=17)
    plt.xlabel(_x, fontsize=10)

# Boxplot: Number of words
plt.subplot(2, 3, 1)
display_boxplot('toxic', 'num_words', train_data, 'No. of words in each class')

# Boxplot: Number of chars
plt.subplot(2, 3, 2)
display_boxplot('toxic', 'num_char', train_data, 'Number of characters in each class')

# Boxplot: Number of unique words
plt.subplot(2, 3, 3)
display_boxplot('toxic', 'num_unique_words', train_data, 'Number of unique words in each class')

# Boxplot: Number of special characters
plt.subplot(2, 3, 4)
display_boxplot('toxic', 'num_special_char', train_data, 'No. of special characters in each class')

# Boxplot: Number of stopwords
plt.subplot(2, 3, 5)
display_boxplot('toxic', 'num_stopwords', train_data, 'Number of stopwords in each class')

# Boxplot: Number of capital letters
plt.subplot(2, 3, 6)
display_boxplot('toxic', 'num_capital_let', train_data, 'No. of capital letters in each class')


plt.subplots_adjust(right=3.0)
plt.subplots_adjust(top=2.0)
plt.show()

Wordcloud of all comments

In [ ]:
def nonan(x):
    if type(x) == str:
        return x.replace("\n", "")
    else:
        return ""

text = ' '.join([nonan(abstract) for abstract in train_data["comment_text"][train_data["toxic"]>0.5]])
wordcloud = WordCloud(max_font_size=None, background_color='black', collocations=False,
                      width=1200, height=1000).generate(text)
fig_1 = px.imshow(wordcloud)
fig_1.update_layout(title_text='Common words in toxic comments')


In [ ]:
def new_len(x):
    if type(x) is str:
        return len(x.split())
    else:
        return 0

train_data["comment_words"] = train_data["comment_text"].apply(new_len)
nums = train_data.query("comment_words != 0 and comment_words < 200").sample(frac=0.1)["comment_words"]
fig = ff.create_distplot(hist_data=[nums],
                         group_labels=["All comments"],
                         colors=["coral"])

fig.update_layout(title_text="Comment words", xaxis_title="Comment words", template="simple_white", showlegend=False)
fig.show()

### remove stop words

In [ ]:
punctuations = string.punctuation
stop_words = set(stopwords.words("english"))

def remove_stop_words(words):
    return [w for w in words if not w in stop_words]

def remove_empty(words):
    return [w for w in words if not len(w.strip())<2]

def words_to_lowercase(words):
    return [w.lower() for w in words]

def remove_punctuation_from_word(word):
    return ''.join(c for c in word if c not in punctuations)

def remove_punctuation_from_words(words):
    return [remove_punctuation_from_word(w) for w in words]

In [ ]:
train_cleaned = train_data_df.copy()

train_cleaned['comment_text'] = train_cleaned['comment_text'].apply(word_tokenize) \
                                                             .apply(words_to_lowercase) \
                                                             .apply(remove_stop_words) \
                                                             .apply(remove_empty)
                                                             #.apply(remove_punctuation_from_words) \

In [ ]:
print(train_cleaned['comment_text'][0:3])

In [ ]:
print(train_cleaned['comment_text'][0:3])

### emojis

In [ ]:
# Function for converting emojis into word
def convert_emojis(words):
    text = ' '.join(words)
    for emot in UNICODE_EMO:
        text = text.replace(emot, "_".join(UNICODE_EMO[emot].replace(",","").replace(":","").split()))
    return text

train_cleaned['comment_text'] = train_cleaned['comment_text'].apply(convert_emojis)

###  abbreviation

In [ ]:
# abbreviation_pipe=AbbreviationDetector(nlp)
# nlp.add_pipe(abbreviation_pipe)
# def replace_acronyms(text):
#     doc=nlp(txt)
#     altered_tok=[tok.text for tok in doc]
#     print(doc._.abbreviations)
#     for abrv in doc._.abbreviations:
#         altered_tok[abrv.start]=str(abrv._.long_form)
#     return(" ".join(altered_tok))

### stemming

In [ ]:
ps = PorterStemmer() 

def stem_words(words):
    return [ps.stem(w) for w in words]

In [ ]:
print(train_cleaned['comment_text'][0:3])

In [ ]:
# train_cleaned['comment_text'] = train_cleaned['comment_text'].apply(stem_words)

In [ ]:
print(train_cleaned['comment_text'][0:3])

### Lemmatization

In [ ]:
# train_cleaned['comment_text'] = train_cleaned['comment_text'].apply(lemmatizer.lemmatize)

In [ ]:
print(train_cleaned['comment_text'][0:3])

## Vectorisation

In [ ]:
train_cleaned.head()

In [ ]:
print(train_cleaned['comment_text'][0:3])

### Count vectorization

In [ ]:
count_vectorizer = CountVectorizer() 
tfidf_vectorizer = TfidfVectorizer()
hashing_vectorizer = HashingVectorizer()

tfidf_vector = tfidf_vectorizer.fit_transform(train_cleaned['comment_text'])
count_vector = count_vectorizer.fit_transform(train_cleaned['comment_text']) 
hashing_vector = hashing_vectorizer.fit_transform(train_cleaned['comment_text']) 

# print("Vocabulary: ", count_vectorizer.vocabulary_) 
# print("Vocabulary: ", tfidf_vectorizer.vocabulary_)

In [ ]:
x_df = train_cleaned.copy()
x_df["tfidf_vector"]= tfidf_vector
x_df["count_vector"]= count_vector
x_df["hashing_vector"]= hashing_vector
y_df = train_cleaned.toxic

In [ ]:
x_train_1, x_test_1, y_train_1, y_test_1 = train_test_split(tfidf_vector, y_df, test_size=0.3, random_state=42)
x_train_2, x_test_2, y_train_2, y_test_2 = train_test_split(count_vector, y_df, test_size=0.3, random_state=42)
x_train_3, x_test_3, y_train_3, y_test_3 = train_test_split(hashing_vector, y_df, test_size=0.3, random_state=42)

In [ ]:
lg_regression_classifier_tfidf = LogisticRegression().fit(x_train_1, y_train_1)
lg_regression_classifier_count = LogisticRegression().fit(x_train_2, y_train_2)
lg_regression_classifier_hashing = LogisticRegression().fit(x_train_3, y_train_3)

multinomial_nb_classifier_tfidf = MultinomialNB().fit(x_train_1, y_train_1)
multinomial_nb_classifier_count = MultinomialNB().fit(x_train_2, y_train_2)
#multinomial_nb_classifier_hashing = MultinomialNB().fit(x_train_3, y_train_3)

In [ ]:
print('lg_regression_classifier_tfidf : ',lg_regression_classifier_tfidf.score(x_test_1, y_test_1))
print('lg_regression_classifier_count : ',lg_regression_classifier_count.score(x_test_2, y_test_2))
print('lg_regression_classifier_hashing : ',lg_regression_classifier_hashing.score(x_test_3, y_test_3))

print('multinomial_nb_classifier_tfidf : ',multinomial_nb_classifier_tfidf.score(x_test_1, y_test_1))
print('multinomial_nb_classifier_count : ',multinomial_nb_classifier_count.score(x_test_2, y_test_2))
#print('multinomial_nb_classifier_hashing : ',clf_regression.score(x_test_3, y_test_3))

In [ ]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev


In [ ]:
import os
import torch
import pandas as pd
from scipy import stats
import numpy as np
import pandas as pd

from tqdm import tqdm
from collections import OrderedDict, namedtuple
import torch.nn as nn
from torch.optim import lr_scheduler
import joblib

import logging
import transformers
import sys



In [ ]:
!pip uninstall transformers  -y
!pip install transformers==2.1.0

In [ ]:
import transformers

print(transformers.__version__)

In [ ]:

class BERTBaseUncased(nn.Module):
    def __init__(self, bert_path):
        super(BERTBaseUncased, self).__init__()
        self.bert_path = bert_path
        self.bert = transformers.BertModel.from_pretrained(self.bert_path)
        self.bert_drop = nn.Dropout(0.3)
        self.out = nn.Linear(768 * 2, 1)

    def forward(
            self,
            ids,
            mask,
            token_type_ids
    ):
        o1, o2 = self.bert(
            ids,
            attention_mask=mask,
            token_type_ids=token_type_ids)
        
        apool = torch.mean(o1, 1)
        mpool, _ = torch.max(o1, 1)
        cat = torch.cat((apool, mpool), 1)

        bo = self.bert_drop(cat)
        p2 = self.out(bo)
        return p2


class BERTDatasetTest:
    def __init__(self, comment_text, tokenizer, max_length):
        self.comment_text = comment_text
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.comment_text)

    def __getitem__(self, item):
        comment_text = str(self.comment_text[item])
        comment_text = " ".join(comment_text.split())

        inputs = self.tokenizer.encode_plus(
            comment_text,
            None,
            add_special_tokens=True,
            max_length=self.max_length,
        )
        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        mask = inputs["attention_mask"]
        
        padding_length = self.max_length - len(ids)
        
        ids = ids + ([0] * padding_length)
        mask = mask + ([0] * padding_length)
        token_type_ids = token_type_ids + ([0] * padding_length)
        
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long)
        }
df = pd.read_csv("../input/jigsaw-multilingual-toxic-comment-classification/test.csv")

tokenizer = transformers.BertTokenizer.from_pretrained("../input/bert-base-multilingual-uncased/", do_lower_case=True)

device = "cuda"
model = BERTBaseUncased(bert_path="../input/bert-base-multilingual-uncased/").to(device)
model.load_state_dict(torch.load("../input/jbertml/model.bin"))
model.eval()

valid_dataset = BERTDatasetTest(
        comment_text=df.content.values,
        tokenizer=tokenizer,
        max_length=192
)

valid_data_loader = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=64,
    drop_last=False,
    num_workers=4,
    shuffle=False
)

with torch.no_grad():
    fin_outputs = []
    for bi, d in tqdm(enumerate(valid_data_loader)):
        ids = d["ids"]
        mask = d["mask"]
        token_type_ids = d["token_type_ids"]

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)

        outputs = model(
            ids=ids,
            mask=mask,
            token_type_ids=token_type_ids
        )

        outputs_np = outputs.cpu().detach().numpy().tolist()
        fin_outputs.extend(outputs_np)
        
sample = pd.read_csv("../input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv")
sample.loc[:, "toxic"] = fin_outputs
sample.to_csv("submission.csv", index=False)